## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-16-20-49-33 +0000,startribune,What we learned from the Vikings’ 20-12 presea...,https://www.startribune.com/minnesota-vikings-...
1,2025-08-16-20-46-32 +0000,nypost,Hochul throws shade on Mamdani’s NYC-owned gro...,https://nypost.com/2025/08/16/us-news/hochul-t...
2,2025-08-16-20-35-01 +0000,startribune,Trump reverses on the need for a ceasefire bef...,https://www.startribune.com/trump-speaks-to-uk...
3,2025-08-16-20-32-44 +0000,startribune,COVID is rising. New vaccines may not be ready...,https://www.startribune.com/covid-is-rising-ne...
4,2025-08-16-20-29-45 +0000,nyt,Trump Backs Plan to Cede Land for Peace in Ukr...,https://www.nytimes.com/live/2025/08/16/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2333/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
17,trump,65
62,putin,42
27,ukraine,24
81,summit,13
240,meeting,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
168,2025-08-16-00-14-50 +0000,nypost,"Putin tries to woo Trump at Alaska meeting, cl...",https://nypost.com/2025/08/15/us-news/putin-tr...,179
35,2025-08-16-18-48-22 +0000,nypost,How Trump’s meeting with Putin pivoted from Uk...,https://nypost.com/2025/08/16/us-news/inside-h...,178
127,2025-08-16-08-01-18 +0000,nyt,Trump and Putin had an isolated conversation i...,https://www.nytimes.com/live/2025/08/15/world/...,177
156,2025-08-16-01-50-10 +0000,bbc,"No ceasefire, no deal: What summit means for T...",https://www.bbc.com/news/articles/clyvd3gkg1po...,177
151,2025-08-16-02-30-31 +0000,nypost,Trump’s ‘quiet’ stance after Alaska meeting wi...,https://nypost.com/2025/08/15/us-news/trumps-q...,172


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
168,179,2025-08-16-00-14-50 +0000,nypost,"Putin tries to woo Trump at Alaska meeting, cl...",https://nypost.com/2025/08/15/us-news/putin-tr...
10,57,2025-08-16-20-15-27 +0000,wapo,Canada orders Air Canada flight attendants bac...,https://www.washingtonpost.com/world/2025/08/1...
19,48,2025-08-16-19-41-53 +0000,nypost,Trump’s efforts at Alaska summit applauded by ...,https://nypost.com/2025/08/16/world-news/trump...
56,40,2025-08-16-16-37-41 +0000,nypost,Zelensky warns Putin will step up attacks on U...,https://nypost.com/2025/08/16/world-news/zelen...
51,38,2025-08-16-17-06-24 +0000,nypost,NYPD drone allows Mayor Adams to see real-time...,https://nypost.com/2025/08/16/us-news/drone-al...
41,30,2025-08-16-18-14-20 +0000,wapo,West Virginia to send National Guard troops to...,https://www.washingtonpost.com/nation/2025/08/...
200,29,2025-08-15-20-58-21 +0000,nypost,Trump reacts to Hillary Clinton saying she’d n...,https://nypost.com/2025/08/15/us-news/trump-re...
47,27,2025-08-16-18-00-00 +0000,nypost,Greedy landlords — not Bronx residents — were ...,https://nypost.com/2025/08/16/us-news/greedy-l...
35,24,2025-08-16-18-48-22 +0000,nypost,How Trump’s meeting with Putin pivoted from Uk...,https://nypost.com/2025/08/16/us-news/inside-h...
54,24,2025-08-16-16-47-52 +0000,nypost,Long Island mall scion in legal dog fight with...,https://nypost.com/2025/08/16/us-news/long-isl...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
